In [1]:
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback

# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)		# 显示行数
pd.set_option('display.max_columns', 500)	# 显示列数
pd.set_option('display.width', 1000)		# 显示宽度

worthDir = '../pkl'  # '/home/at/ipynb/pd' ,  '/home/at/py/ipynb/'
dbfile = '%s/%s' % (worthDir, 'binance.db') # '/mnt/c/milu/docx/strategy/v2.1/binance' ,  '/mnt/e/milu/py'

serverTimeadj = 10
locTimeadj = 0 # 28800  0
newClientOrderId = ''
orderId = ''
name = 'sim'
curDate = time.strftime("%Y%m%d")
timestamp = int(round((time.time() ) * 1000) )

serverUrl = 'http://testnet.binancefuture.com'
# serverUrl = 'http://www.binance.com'
# serverUrl = 'https://fapi.binance.com'

#  #ation126@hotmail.com
# api_key    = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"  # 密钥
# secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"  # 私钥

# #binance@snapmail.cc Test2022
# api_key     = "d7188e86617bec4e95daa012fbaabf4a84ce1b12992019b7125df19eb536ad42"  # 密钥
# secret_key  = "cb68c9cf4303fc4cd11da655df3a978d41fcbf3b1e927f6e6d49ef41388a6749"  # 私钥

# testnet@snapmail.cc  Test2022 15,000.00000000
# api_key    = "40642311a14066133c8076db5472bfff3683d3d1c73e48ae4c6bfc0b405b1957"  # 密钥 API 密钥
# secret_key = "dcff08b6bdbf37caa606e3c6eaa11c2d59d5dcb5f7b1cf7c959aa123e463a43b"  # 私钥 API 私钥

# testnet1@snapmail.cc  Test2022 15,000.00000000
# api_key    = "056624748cc8a9a8c813908172c3e76e8cbd54a8c6d39f5f43fdfead0cae866f"  # 密钥 API 密钥
# secret_key = "b721938f7817a449f7869ac4f49b2f25c6fb5a89809a5319528824f8e75e9cd2"  # 私钥 API 私钥

# testnet2@snapmail.cc  Test2022 15,000.00000000
# api_key    = "b183605895cec96614b1f5cfd455458c7a5c74a01f28d2a33ecda09de84eb0a4"  # 密钥 pubkey
# secret_key = "936fdaa1df30fba4cb8551439487beb56b4728d67c82a8e2b730a8b54ef361dd"  # 私钥 prikey

# testnet3@snapmail.cc  Test2022 15,000.00000000
# api_key    = "067d5cc151564fa0f0c89f81d1ee735aa84299ac1cf5c57c430844745954d2cf"  # 密钥 pubkey
# secret_key = "e590c6caeaa7ce53a41a8c81bdf33992d1057ecf9b5cb30474a83afe51bdbaa8"  # 私钥 prikey

# #oCAuFzSgAumY Real2022
# api_key     = "T6qUmdYPmqwfHOo3ttCJilZMYyxkdaDGROhJ8fZhgmGgTEagXsnXLsXeCUW7LdXG"  # 密钥
# secret_key  = "oDJzCvyN9BDfyIgUc2dibgYeadxQxH9afu4peYuIyfa4sOFOwLQ57Rqui9N95gzJ"  # 私钥

# testnet28@snapmail.cc  Test2022 15,000  *99.13-risk3-fee1-subfee1-testfee1-
api_key    = "7dc31abfddc161ee6437cee02c0e955295c3fe16bc0505a0c808feab500e394f"  # 密钥 pubkey
secret_key = "7e763fef7d6eeee9a8d683fb5d9711464b9a996ebae0fef10b26c66c8be2c3b2"  # 私钥 prikey

recvWindow = '5000'
g_access = 'similarity'

url = f'{serverUrl}/fapi/v1/time'
res = requests.get(url)
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    serverTime = rt['serverTime']
    serverTimeadj = ((rt['serverTime']- timestamp) ) //1000

df_balance = pd.DataFrame()
df_account = pd.DataFrame()
df_position = pd.DataFrame()
df_posirisk = pd.DataFrame()
df_order = pd.DataFrame()
df_trade = pd.DataFrame()

version = "1.0.1"

print(f"{sys.version} , {sys.executable} , pandas: {pd.__version__}")
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****import**v{version}*** ')

3.10.4 (main, Feb 15 2023, 15:33:42) [GCC 9.4.0] , /home/codespace/.python/current/bin/python3 , pandas: 1.5.3
2023-03-18 07:16:12 *****import**v1.0.1*** 


In [ ]:
# #ation126@hotmail.com
api_key    = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"  # 密钥
secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"  # 私钥

# #binance@snapmail.cc Test2022
# api_key     = "d7188e86617bec4e95daa012fbaabf4a84ce1b12992019b7125df19eb536ad42"  # 密钥
# secret_key  = "cb68c9cf4303fc4cd11da655df3a978d41fcbf3b1e927f6e6d49ef41388a6749"  # 私钥


# testnet1@snapmail.cc  Test2022 15,000  *68.31-real-bintest-1
# api_key    = "056624748cc8a9a8c813908172c3e76e8cbd54a8c6d39f5f43fdfead0cae866f"  # 密钥 API 密钥
# secret_key = "b721938f7817a449f7869ac4f49b2f25c6fb5a89809a5319528824f8e75e9cd2"  # 私钥 API 私钥

# testnet2@snapmail.cc  Test2022 15,000  *99.13-test-bintest-2
# api_key    = "b183605895cec96614b1f5cfd455458c7a5c74a01f28d2a33ecda09de84eb0a4"  # 密钥 pubkey
# secret_key = "936fdaa1df30fba4cb8551439487beb56b4728d67c82a8e2b730a8b54ef361dd"  # 私钥 prikey

# testnet3@snapmail.cc  Test2022 15,000  *99.13-test-bintest-3
# api_key    = "067d5cc151564fa0f0c89f81d1ee735aa84299ac1cf5c57c430844745954d2cf"  # 密钥 pubkey
# secret_key = "e590c6caeaa7ce53a41a8c81bdf33992d1057ecf9b5cb30474a83afe51bdbaa8"  # 私钥 prikey

# testnet4@snapmail.cc  Test2022  1,000  *99.13-test-bintest-4
# api_key    = "ab07d48c01650f26c5fecd86c6d22b6b17a2425dd1307267d8615e1402f368db"  # 密钥 API 密钥
# secret_key = "fef0312fe7cbe517c7ad25b77ce26ac9bf54932eb3a1f1242999ba211d882f3a"  # 私钥 API 私钥

# #oCAuFzSgAumY Real2022
# api_key     = "T6qUmdYPmqwfHOo3ttCJilZMYyxkdaDGROhJ8fZhgmGgTEagXsnXLsXeCUW7LdXG"  # 密钥
# secret_key  = "oDJzCvyN9BDfyIgUc2dibgYeadxQxH9afu4peYuIyfa4sOFOwLQ57Rqui9N95gzJ"  # 私钥

"pubkey": "deaa3d08f18e39ecfb35afd97c4e3b4ada48141ee9f4d1879550412ca23b0c56",
"prikey": "fad8feea046b775abb1a12877c868a7bac1526b1637044796b562f8dc73d1ffa",


# testnet11@snapmail.cc  Test2022 15,000  *99.13-risk-risk1-
# api_key    = "8fe7623f5b17d916e19a2d04e8d6e7a65761daf678c1a90120fb69055a28f748"  # 密钥 pubkey
# secret_key = "1770a8f54c937fa86c8d5c2155250f558531fdd6482c189e9881f8596b454662"  # 私钥 prikey

# testnet12@snapmail.cc  Test2022 15,000  *99.13-kline-kline1-
# api_key    = "9dcb00b5339f33c7138188322b6c5f45d74f5b081d6fa50d5ed97df7bb420432"  # 密钥 pubkey
# secret_key = "0887a29bca1961376dfe50d2efd51989698f982668c568d3dfa0408966a2ed7a"  # 私钥 prikey

# testnet13@snapmail.cc  Test2022 15,000  *99.13-risk-order1-
# api_key    = "d1322cbc05610f354531eac6727a6475977c83f713ebe0cb3635fd3c6a2b49ca"  # 密钥 pubkey
# secret_key = "4edf2bf70514120039fed291435cc65f77961fc164d26e72037799532d774c5e"  # 私钥 prikey

# testnet14@snapmail.cc  Test2022 15,000  *99.13-risk-trade1-
# api_key    = "875e36221bb0ebdd847ea4ab64f56114eeae56daf1fcb29116c7ee41d0794394"  # 密钥 pubkey
# secret_key = "a06b6e839a42b527733fafa051e5c1acd240419dac3801079c6d0c5ddfd5b025"  # 私钥 prikey

# testnet15@snapmail.cc  Test2022 15,000  *99.13-risk-risk5-
# api_key    = "6433dff9990704302ede50ca608edfd27b22866cacf69f800fe8eaca5c241811"  # 密钥 pubkey
# secret_key = "d2b2bdf0485ab4fb20d9aab7ae7754b993264d26859bcf541eba4289769956e0"  # 私钥 prikey

# testnet16@snapmail.cc  Test2022 15,000  *99.13-risk-risk6-
# api_key    = "2d176ba955cff6ae618159186753761e2a1e2ec705f818e1d8bd68a7c46343c6"  # 密钥 pubkey
# secret_key = "69327db724b70c99d9e0e9eafa1f34110a44f1a583a384e7beb379697e2d1ea2"  # 私钥 prikey

# testnet17@snapmail.cc  Test2022 15,000  *99.13-risk-risk7-
# api_key    = "465b350908c7850354dd1df47e4a32ec989d928c8e8aa1d15d20f03f4e691b53"  # 密钥 pubkey
# secret_key = "63aba318d49f4a6bb5286226ee8705495eeb5213d308d7d05dc8265a12ce2109"  # 私钥 prikey


# testnet21@snapmail.cc  Test2022 15,000  *99.13-risk2-risk21-
# api_key    = "38c8df06299e955d57e9df4a520bb5589459d457f6ccf5fcced23f1033f151ce"  # 密钥 pubkey
# secret_key = "a1cde020fb28435944bcddb03280e783b58d5764a952f122ea4653b6f2a8383e"  # 私钥 prikey


# testnet22@snapmail.cc  Test2022 15,000  *99.13-order2-order21-
# api_key    = "4975e5ea27e6497db930f6773ac70dd12f05f76753b227125b60715c779ec9d8"  # 密钥 pubkey
# secret_key = "cff76bf806526224454411d7a31e773b6517ae6bcdcd76fdefcd4e37817a9991"  # 私钥 prikey


# testnet23@snapmail.cc  Test2022 15,000  *99.13-risk3-kline1-
# api_key    = "598e0636f68883477483b545ba086231e1be63458248e196c7b837820a6e498b"  # 密钥 pubkey
# secret_key = "a633ce2b580dd5b1c578247340dbae619e566a0217d79b5ab6008b40de0365fd"  # 私钥 prikey


# testnet24@snapmail.cc  Test2022 15,000  *99.13-risk3-kline2-
# api_key    = "c8e4db02d4339972299ff5423b0b580d87d558d40d8262e12a2a45efd6a8a271"  # 密钥 pubkey
# secret_key = "ee0a42f9459676021f50ce301b7cc93b7101a92e40224b40555bff1aa45625e2"  # 私钥 prikey


# testnet25@snapmail.cc  Test2022 15,000  *99.13-risk3-kline3-
# api_key    = "76632e802c8da3c40aec0f15b29af406e7584a2d273e46cf72603a5e6a265d75"  # 密钥 pubkey
# secret_key = "16662ef22fddede042bcf7be2f4297d1b451da7443dded5e11732f84a4487337"  # 私钥 prikey


# testnet26@snapmail.cc  Test2022 15,000  *99.13-risk3-kline4-
# api_key    = "968532305fa5c74686a3c8980ed560783afd13cc07f6a27763df1fbb6e997428"  # 密钥 pubkey
# secret_key = "9579a4afaa29514fc37e76779f1506879e535ae3d4e5a410aec0bdfa4103dfdd"  # 私钥 prikey


# testnet27@snapmail.cc  Test2022 15,000  *99.13-risk3-order1-
# api_key    = "deeec9c07260e1a0f61f76e69ee43c431a95716ff60178b24472ab707409030a"  # 密钥 pubkey
# secret_key = "a36f9f4c78ff5a5d4f2599ed9e3e31f4297271021ba908857cf3269be8711188"  # 私钥 prikey

# testnet28@snapmail.cc  Test2022 15,000  *99.13-risk3-fee1-subfee1-testfee1-
# api_key    = "7dc31abfddc161ee6437cee02c0e955295c3fe16bc0505a0c808feab500e394f"  # 密钥 pubkey
# secret_key = "7e763fef7d6eeee9a8d683fb5d9711464b9a996ebae0fef10b26c66c8be2c3b2"  # 私钥 prikey


# testnet31@snapmail.cc  Test2022 15,000  *68.31-testrisk1-testnet31_risk1-
# api_key    = "f9529730010f598de9b0d4a9b0fe156b5541aba60f979f0e5c8902a6f830892f"  # 密钥 pubkey
# secret_key = "dd53d75eaca841596ecb86429e1f1f2d62a433de1d7ab819d69636943803191e"  # 私钥 prikey

# testnet32@snapmail.cc  Test2022 15,000  *68.31-testrisk1-testnet32_kline1-
# api_key    = "0f9cba70f810787ad4d3cd6c49c8f97f14fca5f53bcd59a487e38c8eeed36a7b"  # 密钥 pubkey
# secret_key = "49371d0e73eb89eef8d30c4a49b8d826d446b7d4aba75f652f2404cac165ee6d"  # 私钥 prikey



In [14]:
# binance.db

import os
import sqlite3
import pandas as pd

worth_real_dbfile = f"/mnt/d/milu/bak/worth_real.db"  # worth.db

if os.path.exists(worth_real_dbfile):
    # print(f"{worth_real_dbfile}  exists ")

    conn = sqlite3.connect(worth_real_dbfile)

    df_balance = pd.read_sql(' select * from balance ', conn) 
    # access accountAlias asset availableBalance balance crossUnPnl crossWalletBalance marginAvailable maxWithdrawAmount queryTime updateDate updateTime
    # print(f"df_balance.iloc[-5:,:] :\n {df_balance.iloc[-5:,:]} \n")
    df_b = df_balance[['access','asset','balance','availableBalance','updateTime','queryTime', ] ]
    print(f"df_b.iloc[-5:,:] :\n {df_b.iloc[-5:,:]} \n")

    df_orders = pd.read_sql(' select * from orders ', conn)
    # access avgPrice clientOrderId closePosition cumQuote executedQty orderId origQty origType positionSide price priceProtect queryTime reduceOnly side status stopPrice symbol time timeInForce type updateDate updateTime workingType
    # print(f"df_orders.iloc[-5:,:] :\n {df_orders.iloc[-5:,:]} \n")
    df_o = df_orders[['access','orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]
    print(f"df_o.iloc[-5:,:] :\n {df_o.iloc[-5:,:]} \n")

    df_posirisk = pd.read_sql(' select * from posirisk ', conn) 
    # access entryPrice isAutoAddMargin isolatedMargin isolatedWallet leverage liquidationPrice marginType markPrice maxNotionalValue notional positionAmt positionSide queryTime symbol unRealizedProfit updateDate updateTime
    # print(f"df_posirisk.iloc[-5:,:] :\n {df_posirisk.iloc[-5:,:]} \n")
    df_pr = df_posirisk[['access','symbol','markPrice','unRealizedProfit','leverage','entryPrice','positionAmt','notional','updateDate','queryTime', ] ]
    print(f"df_pr.iloc[-5:,:] :\n {df_pr.iloc[-5:,:]} \n")

    df_trades = pd.read_sql(' select * from trades ', conn) 
    # access buyer commission commissionAsset id maker marginAsset orderId positionSide price qty queryTime quoteQty realizedPnl side symbol time updateDate
    # print(f"df_trades.iloc[-5:,:] :\n {df_trades.iloc[-5:,:]} \n")
    df_tr = df_trades[['access','id','orderId','symbol','price','qty','side','positionSide','quoteQty', 'queryTime'] ]
    print(f"df_tr.iloc[-5:,:] :\n {df_tr.iloc[-5:,:]} \n")

    # df_a = df_account[['asset','walletBalance','availableBalance','maintMargin','marginBalance','unrealizedProfit','updateDate','queryTime', ] ]

    # rt_df.to_sql('account', con=conn, if_exists='append', index=False) #  replace替换 append追加
    conn.close()

# df_account.to_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
# # df_acc = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
# nohup /usr/bin/python3 updateBalance.py -n similarity >> log.txt 2>&1  &

df_b.iloc[-5:,:] :
                access asset        balance  availableBalance     updateTime            queryTime
30828  similarity_big  USDT  190850.038012     190850.038012  1674123100190  2023-01-21 06:55:06
30829      similarity  USDT    1008.878174       1008.878174  1674230403809  2023-01-21 07:00:01
30830  similarity_big  USDT  190811.873072     143109.045729  1674255605435  2023-01-21 07:00:06
30831      similarity  USDT    1008.480530        509.142853  1674255605435  2023-01-21 07:05:02
30832  similarity_big  USDT  190811.873072     142856.038452  1674255605435  2023-01-21 07:05:06 

df_o.iloc[-5:,:] :
             access       orderId   symbol     price origQty  status                  updateDate                         clientOrderId
50  similarity_big  109449120860  BTCUSDT     22000   0.001     NEW  2023-01-20 07:52:42.265000              ios_syWMeb7UFi594rSvu49b
51  similarity_big  109479554433  BTCUSDT  88888.80   0.001     NEW  2023-01-20 09:53:19.855000  SIS20230120

In [5]:
###binancefuture 账号api#####
d1 = {    
    "account_type" : "binancefuture",
    "account_name" : "Test2022",
    "api_key" : "d7188e86617bec4e95daa012fbaabf4a84ce1b12992019b7125df19eb536ad42",
    "secret_key" : "cb68c9cf4303fc4cd11da655df3a978d41fcbf3b1e927f6e6d49ef41388a6749",
    "accountAlias": "FzFzXqfWSgSgoC",
    "leverage": 2
}

d2 = {    
    "account_type" : "binancefuture",
    "account_name" : "at",
    "api_key" : "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e",
    "secret_key" : "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876",
    "accountAlias": "FzuXfWTioCmYmY",
    "leverage": 2
}

r1 = {    
    "account_type" : "binance",
    "account_name" : "Real2022",
    "api_key" : "T6qUmdYPmqwfHOo3ttCJilZMYyxkdaDGROhJ8fZhgmGgTEagXsnXLsXeCUW7LdXG",
    "secret_key" : "oDJzCvyN9BDfyIgUc2dibgYeadxQxH9afu4peYuIyfa4sOFOwLQ57Rqui9N95gzJ",
    "accountAlias": "oCAuFzSgAumY",
    "leverage": 2
}


print(f"{ -1e-3 } ")

-0.001 


In [2]:
###服务器时间 QryServerTime##20220919A######
import requests
import json
import time

def QryServerTime():
    url = f'{serverUrl}/fapi/v1/time'
    res = requests.get(url)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else:
        srvTime = rt['serverTime']  
    
    return srvTime  
    
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****QryServerTime***** ')

2023-03-18 07:16:48 *****QryServerTime***** 


In [5]:
###日志 logger##20220825A######
import logging
import sys
import time

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')

def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    logger = logging.getLogger(name)
    if not logger.handlers:
        logger.setLevel(level=log_level)
        handler = logging.FileHandler(log_file, mode=file_mode)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(level=log_level)
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    return logger


logger = make_logger(name, log_level=logging.DEBUG, log_file= "../logs/%s_%s.log" % (name if ('name' in dir() ) else 'similarity', curDate if ('curDate' in dir() ) else time.strftime("%Y%m%d") ) )

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****logging***** ')

2023-03-18 07:18:05 *****logging***** 


In [6]:
###查询账户余额 balance###QryBalance###
def QryBalance():

    global g_access, df_balance
    global serverUrl, api_key, secret_key, recvWindow

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )  # 时间戳ms
    url = f'{serverUrl}/fapi/v2/balance'

    query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"recvWindow":recvWindow, "timestamp":timestamp, "signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url, headers=headers, params=params)
    rt = json.loads(res.text)
    logger.info(f'params: {params} \n rt: {rt}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res: {res.text}')
    else :
        for bal in rt:
            if bal['asset'] != 'USDT':
                continue

            rt_df = pd.DataFrame([bal])

            rt_df = pd.DataFrame([bal])        
            # rt_df = pd.DataFrame([rt[1] ])
            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000)) 
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00")
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
            rt_df['balance']  = pd.to_numeric(rt_df['balance'] , errors='coerce', downcast='float')
            rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
            rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
            rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')
            rt_df['access'] = g_access

            rt_df.sort_index(axis=1, ascending=True, inplace=True)

            df_balance = pd.concat([df_balance, rt_df ], ignore_index=True)
            df_b = df_balance[['asset','balance','availableBalance','updateTime','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_balance = pd.read_sql(' select * from balance ', conn) 
            rt_df.to_sql('balance', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_balance.to_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )
            # df_bal = pd.read_pickle('%s/%s.pkl' % (worthDir, "balancedf" ) )

            logger.info(f'df_b :\n{df_b}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询账户余额 balance###QryBalance**v{version}*** ')

2023-03-18 07:18:26 *****查询账户余额 balance###QryBalance**v1.0.1*** 


In [7]:
###查询账户数据v2 account###余额assets+持仓positions###QryAccount###
def QryAccount():

    global serverUrl, api_key, secret_key, recvWindow
    global g_access, df_account, df_position

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )

    url = f'{serverUrl}/fapi/v2/account'

    query_string = 'recvWindow=%s&timestamp=%s' % (recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    # logger.info(f'res.text :\n{res.text}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :

        for ass in rt["assets"]:

            if ass['asset'] != 'USDT':
                continue 

            rt_df = pd.DataFrame([ass])

            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            rt_df['availableBalance']  = pd.to_numeric(rt_df['availableBalance'] , errors='coerce', downcast='float')
            rt_df['crossUnPnl']  = pd.to_numeric(rt_df['crossUnPnl'] , errors='coerce', downcast='float')
            rt_df['crossWalletBalance']  = pd.to_numeric(rt_df['crossWalletBalance'] , errors='coerce', downcast='float')
            rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
            rt_df['marginBalance']  = pd.to_numeric(rt_df['marginBalance'] , errors='coerce', downcast='float')
            rt_df['maxWithdrawAmount']  = pd.to_numeric(rt_df['maxWithdrawAmount'] , errors='coerce', downcast='float')
            rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
            rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
            rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
            rt_df['walletBalance']  = pd.to_numeric(rt_df['walletBalance'] , errors='coerce', downcast='float')
            rt_df['access'] = g_access

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            df_account = pd.concat([df_account,rt_df ], ignore_index=True)
            df_a = df_account[['asset','walletBalance','availableBalance','maintMargin','marginBalance','unrealizedProfit','updateDate','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_account = pd.read_sql(' select * from account ', conn) 
            rt_df.to_sql('account', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_account.to_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )
            # df_acc = pd.read_pickle('%s/%s.pkl' % (worthDir, "accountdf" ) )

            logger.info(f'df_a :\n{df_a}')

        for posi in rt["positions"]:

            if not abs(float(posi['positionAmt'])) > 0 :
                continue 

            rt_df = pd.DataFrame([posi])

            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            rt_df['initialMargin']  = pd.to_numeric(rt_df['initialMargin'] , errors='coerce', downcast='float')
            rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
            rt_df['unrealizedProfit']  = pd.to_numeric(rt_df['unrealizedProfit'] , errors='coerce', downcast='float')
            rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
            rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
            rt_df['entryPrice']  = pd.to_numeric(rt_df['entryPrice'] , errors='coerce', downcast='float')
            rt_df['maxNotional']  = pd.to_numeric(rt_df['maxNotional'] , errors='coerce', downcast='float')
            rt_df['bidNotional']  = pd.to_numeric(rt_df['bidNotional'] , errors='coerce', downcast='float')
            rt_df['askNotional']  = pd.to_numeric(rt_df['askNotional'] , errors='coerce', downcast='float')
            rt_df['positionAmt']  = pd.to_numeric(rt_df['positionAmt'] , errors='coerce', downcast='float')
            rt_df['access'] = g_access
            
            rt_df['leverage']  = pd.to_numeric(rt_df['leverage'] , errors='coerce', downcast='integer')

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            df_position = pd.concat([df_position, rt_df ], ignore_index=True)
            df_p = df_position[['symbol','initialMargin','maintMargin','unrealizedProfit','positionInitialMargin','openOrderInitialMargin','leverage','entryPrice','maxNotional','positionAmt','notional','updateDate','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_position = pd.read_sql(' select * from position ', conn) 
            rt_df.to_sql('position', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_position.to_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )
            # df_posi = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )

            logger.info(f'df_p :\n{df_p}')

print(f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询账户数据 account###余额assets+持仓positions###QryAccount**v{version}*** ')        


2023-03-18 07:18:35 *****查询账户数据 account###余额assets+持仓positions###QryAccount**v1.0.1*** 


In [8]:
###交易单向持仓 order###订单###TradeFutureBoth###
def TradeFutureBoth(symbol='btcusdt', side='BUY', price=19000.1, quantity=0.001, orderType='LIMIT', timeInForce='GTC'):
    # , callbackRate=5, stopPrice=20000, activationPrice=20000, reduceOnly='false', priceProtect='false', closePosition='true'

    global serverUrl, api_key, secret_key, recvWindow
    global g_access, df_order, newClientOrderId, orderId

    timestamp = int(round((time.time() +serverTimeadj+1) * 1000) )
    
    url = f'{serverUrl}/fapi/v1/order'

    # symbol = 'btcusdt'  #交易对 
    # side = 'BUY'        #买卖方向 BUY 买入, SELL 卖出
    # orderType = 'LIMIT'      #订单类型 LIMIT限价单, MARKET市价单, STOP止损限价单, STOP_MARKET止损市价单, TAKE_PROFIT止盈限价单, TAKE_PROFIT_MARKET止盈市价单, TRAILING_STOP_MARKET跟踪止损单,
    # price = 19000.1  #委托价格
    # quantity = 0.001  #下单数量
    # timeInForce = 'GTC' #有效方法 GTC-Good Till Cancel 成交为止, IOC-Immediate or Cancel 无法立即成交(吃单)的部分就撤销, FOK-Fill or Kill 无法全部立即成交就撤销, GTX-Good Till Crossing 无法成为挂单方就撤销

    reduceOnly = 'false' #仅减仓 false true 
    workingType = 'CONTRACT_PRICE'  #stopPrice 触发类型: MARK_PRICE(标记价格), CONTRACT_PRICE(合约最新价). 默认 CONTRACT_PRICE
    priceProtect = 'false' #条件单触发保护："TRUE","FALSE", 默认"FALSE". 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    newOrderRespType = "ACK"  #"ACK", "RESULT", 默认 "ACK"

    sideDict = {'BUY':'B', 'SELL':'S', }
    typeDict = {'LIMIT':'LMT', 'MARKET':'MKT', 'STOP':'STP','STOP_MARKET':'STP','TAKE_PROFIT':'TKP','TAKE_PROFIT_MARKET':'TPM','TRAILING_STOP_MARKET':'TSM', }
    reduceOnlyDict = {'false':0, 'true':1,}
    workingTypeDict = {'CONTRACT_PRICE':'C', 'MARK_PRICE':'M',}
    priceProtectDict = {'false':0, 'true':1,}
    newOrderRespTypeDict = {'ACK':'A', 'RESULT':'R',}

    positionSide = 'BOTH' #持仓方向，单向持仓模式下非必填，默认且仅可填BOTH;在双向持仓模式下必填,且仅可选择 LONG 或 SHORT, BOTH 单一持仓方向, LONG 多头(双向持仓下), SHORT 空头(双向持仓下)
    callbackRate = 5 # 追踪止损回调比例，可取值范围[0.1, 5],其中 1代表1% ,仅TRAILING_STOP_MARKET 需要此参数
    stopPrice = 20000   #触发价, 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    closePosition = 'true' #触发后全部平仓，true, false；仅支持STOP_MARKET和TAKE_PROFIT_MARKET；不与quantity合用；自带只平仓效果，不与reduceOnly 合用
    activationPrice = 20000 #追踪止损激活价格，仅TRAILING_STOP_MARKET 需要此参数, 默认为下单当前市场价格(支持不同workingType)

    closePositionDict = {'false':0, 'true':1,}
    positionSideDict = {'BOTH':'B', 'LONG':'L', 'SHORT':'S', }

    tailStr = '%d%s%d%s%d%s' % (reduceOnlyDict[reduceOnly], workingTypeDict[workingType], priceProtectDict[priceProtect], newOrderRespTypeDict[newOrderRespType], closePositionDict[closePosition], positionSideDict[positionSide] )
    symbolStr = '%s-------' % (symbol.lower() )
    tacticName = 'T0'
    newClientOrderId = '%s%s%s%s%s%s%s' % (tacticName, sideDict[side], time.strftime("%Y%m%d%H%M%S"), typeDict[orderType], symbolStr[:7], timeInForce, tailStr )  #自定义订单号 ^[\.A-Z\:/a-z0-9_-]{1,36}$    2+1+14+3 +7+3+6

    # orderType = 'LIMIT'      #订单类型 LIMIT 限价单, MARKET 市价单, STOP 止损限价单, STOP_MARKET 止损市价单, TAKE_PROFIT 止盈限价单, TAKE_PROFIT_MARKET 止盈市价单, TRAILING_STOP_MARKET 跟踪止损单,

    if orderType == 'LIMIT':
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))

    elif orderType == 'MARKET':
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&quantity=%f&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,quantity,recvWindow,str(timestamp))

    else:
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))

    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"price":price,"quantity":quantity,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}

    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    # res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
    res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    logger.info(f'url2 : {url2}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        del rt['cumQty'] #弃用
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')
        rt_df['access'] = g_access

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values:
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        orderId = rt['orderId']

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from order ', conn) 
        rt_df.to_sql('order', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')
        
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****交易单向持仓 order###订单###TradeFuture***** ')        


2023-03-18 07:18:37 *****交易单向持仓 order###订单###TradeFuture***** 


In [9]:
###交易 order###订单###TradeFuture###
def TradeFuture(symbol='btcusdt', side='BUY', price=19000.1, quantity=0.001, orderType='LIMIT', timeInForce='GTC', positionSide='BOTH'):
    # , callbackRate=5, stopPrice=20000, activationPrice=20000, reduceOnly='false', priceProtect='false', closePosition='true'

    global serverUrl, api_key, secret_key, recvWindow
    global g_access, df_order, newClientOrderId, orderId

    timestamp = int(round((time.time() +serverTimeadj+1) * 1000) )
    
    url = f'{serverUrl}/fapi/v1/order'

    # symbol = 'btcusdt'  #交易对 
    # side = 'BUY'        #买卖方向 BUY 买入, SELL 卖出
    # orderType = 'LIMIT'      #订单类型 LIMIT限价单, MARKET市价单, STOP止损限价单, STOP_MARKET止损市价单, TAKE_PROFIT止盈限价单, TAKE_PROFIT_MARKET止盈市价单, TRAILING_STOP_MARKET跟踪止损单,
    # price = 19000.1  #委托价格
    # quantity = 0.001  #下单数量
    # timeInForce = 'GTC' #有效方法 GTC-Good Till Cancel 成交为止, IOC-Immediate or Cancel 无法立即成交(吃单)的部分就撤销, FOK-Fill or Kill 无法全部立即成交就撤销, GTX-Good Till Crossing 无法成为挂单方就撤销

    reduceOnly = 'false' #仅减仓 false true 
    workingType = 'CONTRACT_PRICE'  #stopPrice 触发类型: MARK_PRICE(标记价格), CONTRACT_PRICE(合约最新价). 默认 CONTRACT_PRICE
    priceProtect = 'false' #条件单触发保护："TRUE","FALSE", 默认"FALSE". 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    newOrderRespType = "ACK"  #"ACK", "RESULT", 默认 "ACK"

    sideDict = {'BUY':'B', 'SELL':'S', }
    typeDict = {'LIMIT':'LMT', 'MARKET':'MKT', 'STOP':'STP','STOP_MARKET':'STP','TAKE_PROFIT':'TKP','TAKE_PROFIT_MARKET':'TPM','TRAILING_STOP_MARKET':'TSM', }
    reduceOnlyDict = {'false':0, 'true':1,}
    workingTypeDict = {'CONTRACT_PRICE':'C', 'MARK_PRICE':'M',}
    priceProtectDict = {'false':0, 'true':1,}
    newOrderRespTypeDict = {'ACK':'A', 'RESULT':'R',}

    # positionSide = 'BOTH' #持仓方向，单向持仓模式下非必填，默认且仅可填BOTH;在双向持仓模式下必填,且仅可选择 LONG 或 SHORT, BOTH 单一持仓方向, LONG 多头(双向持仓下), SHORT 空头(双向持仓下)
    callbackRate = 5 # 追踪止损回调比例，可取值范围[0.1, 5],其中 1代表1% ,仅TRAILING_STOP_MARKET 需要此参数
    stopPrice = 20000   #触发价, 仅 STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET 需要此参数
    closePosition = 'true' #触发后全部平仓，true, false；仅支持STOP_MARKET和TAKE_PROFIT_MARKET；不与quantity合用；自带只平仓效果，不与reduceOnly 合用
    activationPrice = 20000 #追踪止损激活价格，仅TRAILING_STOP_MARKET 需要此参数, 默认为下单当前市场价格(支持不同workingType)

    closePositionDict = {'false':0, 'true':1,}
    positionSideDict = {'BOTH':'B', 'LONG':'L', 'SHORT':'S', }

    tailStr = '%d%s%d%s%d%s' % (reduceOnlyDict[reduceOnly], workingTypeDict[workingType], priceProtectDict[priceProtect], newOrderRespTypeDict[newOrderRespType], closePositionDict[closePosition], positionSideDict[positionSide] )
    symbolStr = '%s-------' % (symbol.lower() )
    tacticName = 'T0'
    newClientOrderId = '%s%s%s%s%s%s%s' % (tacticName, sideDict[side], time.strftime("%Y%m%d%H%M%S"), typeDict[orderType], symbolStr[:7], timeInForce, tailStr )  #自定义订单号 ^[\.A-Z\:/a-z0-9_-]{1,36}$    2+1+14+3 +7+3+6

    # orderType = 'LIMIT'      #订单类型 LIMIT 限价单, MARKET 市价单, STOP 止损限价单, STOP_MARKET 止损市价单, TAKE_PROFIT 止盈限价单, TAKE_PROFIT_MARKET 止盈市价单, TRAILING_STOP_MARKET 跟踪止损单,

    if orderType == 'LIMIT':
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))

    elif orderType == 'MARKET':
        
        if positionSide == 'BOTH': 
        
            query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&quantity=%f&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,quantity,recvWindow,str(timestamp))
        else:
            query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&quantity=%f&positionSide=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,quantity,positionSide,recvWindow,str(timestamp))

    else:
        query_string = 'newClientOrderId=%s&symbol=%s&side=%s&type=%s&price=%f&quantity=%f&timeInForce=%s&recvWindow=%s&timestamp=%s' % (newClientOrderId,symbol,side,orderType,price,quantity,timeInForce,recvWindow,str(timestamp))

    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"newClientOrderId":newClientOrderId,"symbol":symbol,"side":side,"type":orderType,"price":price,"quantity":quantity,"timeInForce":timeInForce, "recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}

    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    # res = requests.post(url, headers=headers, params=params) #, params=params  urlTest  url2 
    res = requests.post(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    logger.info(f'url2 : {url2}')

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        del rt['cumQty'] #弃用
        rt_df = pd.DataFrame([rt])

        rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
        rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
        # rt_df['time'] = timestamp

        rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
        rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
        rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
        rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
        rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
        rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
        # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
        # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')
        rt_df['access'] = g_access

        rt_df.sort_index(axis=1, ascending=True, inplace=True)

        if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and rt['orderId'] in df_order['orderId'].values:
            df_order.sort_index(axis=1, ascending=True, inplace=True)
            df_order.loc[ df_order['orderId'] == rt['orderId'] ] = rt_df.to_numpy()
        else:
            df_order = pd.concat([df_order,rt_df ], ignore_index=True)

        orderId = rt['orderId']

        df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

        conn = sqlite3.connect(dbfile)
        # ##df_order = pd.read_sql(' select * from order ', conn) 
        rt_df.to_sql('order', con=conn, if_exists='append', index=False) #,
        conn.close()

        df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
        # ##df_ord = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_o :\n{df_o}')
        
print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****交易 order###订单###TradeFuture***** ')        


2023-03-18 07:18:40 *****交易 order###订单###TradeFuture***** 


In [10]:
###查询当前全部挂单 openOrders###订单合并###QryOpenOrders###
def QryOpenOrders():

    global serverUrl, api_key, secret_key, recvWindow
    global newClientOrderId, orderId
    global g_access, df_order

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )

    url = f'{serverUrl}/fapi/v1/openOrders'

    symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密

    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.get(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        for order in rt:
            rt_df = pd.DataFrame([order])

            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")
            # rt_df['time'] = timestamp

            rt_df['cumQuote']  = pd.to_numeric(rt_df['cumQuote'] , errors='coerce', downcast='float')
            rt_df['executedQty']  = pd.to_numeric(rt_df['executedQty'] , errors='coerce', downcast='float')
            rt_df['avgPrice']  = pd.to_numeric(rt_df['avgPrice'] , errors='coerce', downcast='float')
            rt_df['origQty']  = pd.to_numeric(rt_df['origQty'] , errors='coerce', downcast='float')
            rt_df['price']  = pd.to_numeric(rt_df['price'] , errors='coerce', downcast='float')
            rt_df['stopPrice']  = pd.to_numeric(rt_df['stopPrice'] , errors='coerce', downcast='float')
            # rt_df['activatePrice']  = pd.to_numeric(rt_df['activatePrice'] , errors='coerce', downcast='float')
            # rt_df['priceRate']  = pd.to_numeric(rt_df['priceRate'] , errors='coerce', downcast='float')
            rt_df['access'] = g_access
            
            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            
            if  not df_order.empty and newClientOrderId in df_order['clientOrderId'].values and orderId in df_order['orderId'].values: 
                df_order.sort_index(axis=1, ascending=True, inplace=True)
                df_order.loc[ df_order['orderId'] == orderId ] = rt_df.to_numpy()
            else:
                df_order = pd.concat([df_order,rt_df ], ignore_index=True)

            df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

            conn = sqlite3.connect(dbfile)
            # ##df_order = pd.read_sql(' select * from order ', conn) 
            rt_df.to_sql('order', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
            # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        # logger.info(f'df_order :\n{df_order}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询当前全部挂单 openOrders###订单合并###QryOpenOrders***** ')  


2023-03-18 07:18:42 *****查询当前全部挂单 openOrders###订单合并###QryOpenOrders***** 


In [11]:
###撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders###
def CancelAllOpenOrders(symbol='btcusdt'):
    global serverUrl, api_key, secret_key, recvWindow, serverTimeadj, g_access

    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    # recvWindow = '5000'

    url = f'{serverUrl}/fapi/v1/allOpenOrders'
    # api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    # secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910
    # origClientOrderId = rt['origClientOrderId'] if ('rt' in dir() and 'origClientOrderId' in rt) else 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    # orderId = 3204222910
    # origClientOrderId = 'T0B20220828182039LMTbtcusdtGTC0C0A1B'

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    secret = secret_key
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密

    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    res = requests.delete(url,headers=headers,params=params)
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        if rt['code'] == '200':
            logger.info(f'res : {res.text}')
        else :
            logger.error(f'res : {res.text}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders***** ')  

2023-03-18 07:18:45 *****撤销全部订单 allOpenOrders###订单合并###CancelAllOpenOrders***** 


In [12]:
###用户持仓风险V2 positionRisk###get###QryPositionRisk###
def QryPositionRisk(symbol='btcusdt'):
    
    global serverUrl, api_key, secret_key, recvWindow
    global g_access, df_posirisk
    
    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    url = f'{serverUrl}/fapi/v1/positionRisk'

    # symbol = "btcusdt"

    query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)

    res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else:
        # logger.info(f'res : {res.text}')
        for posi in rt:

            if not abs(float(posi['positionAmt'])) > 0 :
                continue 

            rt_df = pd.DataFrame([posi])

            # rt_df['systemTime'] = rt_df['updateTime'].apply(lambda x: datetime.fromtimestamp(x/1000))
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

            # #rt_df['initialMargin']  = pd.to_numeric(rt_df['initialMargin'] , errors='coerce', downcast='float')
            # #rt_df['maintMargin']  = pd.to_numeric(rt_df['maintMargin'] , errors='coerce', downcast='float')
            # rt_df['unRealizedProfit']  = pd.to_numeric(rt_df['unRealizedProfit'] , errors='coerce', downcast='float')  # unRealizedProfit  unrealizedProfit
            # #rt_df['positionInitialMargin']  = pd.to_numeric(rt_df['positionInitialMargin'] , errors='coerce', downcast='float')
            # #rt_df['openOrderInitialMargin']  = pd.to_numeric(rt_df['openOrderInitialMargin'] , errors='coerce', downcast='float')
            # rt_df['entryPrice']  = pd.to_numeric(rt_df['entryPrice'] , errors='coerce', downcast='float')
            # #rt_df['maxNotional']  = pd.to_numeric(rt_df['maxNotional'] , errors='coerce', downcast='float')
            # #rt_df['bidNotional']  = pd.to_numeric(rt_df['bidNotional'] , errors='coerce', downcast='float')
            # #rt_df['askNotional']  = pd.to_numeric(rt_df['askNotional'] , errors='coerce', downcast='float')
            # rt_df['positionAmt']  = pd.to_numeric(rt_df['positionAmt'] , errors='coerce', downcast='float')

            # rt_df['leverage']  = pd.to_numeric(rt_df['leverage'] , errors='coerce', downcast='integer')
            rt_df['access'] = g_access

            rt_df.sort_index(axis=1, ascending=True, inplace=True)
            df_posirisk = pd.concat([df_posirisk, rt_df ], ignore_index=True)
            df_pr = df_posirisk[['symbol','markPrice','unRealizedProfit','leverage','entryPrice','positionAmt','notional','updateDate','queryTime', ] ]

            conn = sqlite3.connect(dbfile)
            # df_posirisk = pd.read_sql(' select * from posirisk ', conn) 
            rt_df.to_sql('posirisk', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_posirisk.to_pickle('%s/%s.pkl' % (worthDir, "posiriskdf" ) )
            # df_posi = pd.read_pickle('%s/%s.pkl' % (worthDir, "positiondf" ) )

            logger.info(f'df_pr :\n{df_pr}')



print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****用户持仓风险V2 positionRisk###get###QryPositionRisk***** ')  

2023-03-18 07:18:46 *****用户持仓风险V2 positionRisk###get###QryPositionRisk***** 


In [13]:
###查询所有订单+历史 allOrders###订单合并###get###QryAllOrders###
def QryAllOrders(symbol='btcusdt', limit=10):

    global serverUrl, api_key, secret_key, recvWindow, newClientOrderId
    global g_access, df_order
    
    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    url = f'{serverUrl}/fapi/v1/allOrders'

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910

    # startTime = 0  # 起始时间, 查询时间范围最大不得超过7天, 默认查询最近7天内的数据
    # endTime = 0 # 结束时间
    # limit = 10 # 返回的结果集数量 默认值:500 最大值:1000

    query_string = 'symbol=%s&limit=%s&recvWindow=%s&timestamp=%s' % (symbol,limit,recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"symbol":symbol,"limit":limit,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)
    res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    # print(f'{url2=} \n{res.text}')

    # [{"orderId":3216740836,"symbol":"BTCUSDT","status":"FILLED","clientOrderId":"SIB20220926080700MKTbtcusdtGTC0C0A1B","price":"0","avgPrice":"18772.85409","origQty":"0.159","executedQty":"0.159","cumQuote":"2984.88380","timeInForce":"GTC","type":"MARKET","reduceOnly":false,"closePosition":false,"side":"BUY","positionSide":"BOTH","stopPrice":"0","workingType":"CONTRACT_PRICE","priceProtect":false,"origType":"MARKET","time":1664150820741,"updateTime":1664150820741},

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        for order in rt:
            rt_df = pd.DataFrame([order])
            # print(f"{rt_df = }  ")

            # rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms") 
            rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms", origin="1970-01-01 08:00:00") 
            # rt_df['queryTime'] = time.strftime("%Y-%m-%d %H:%M:%S")
            rt_df['time'] = timestamp
            rt_df['access'] = g_access
            
            rt_df.sort_index(axis=1, ascending=True, inplace=True)

            # print(f"{ rt_df['orderId'] = } , { rt_df['orderId'][0] = }")
            # print(f"{ df_order['orderId'].values = }")
            
            if  not df_order.empty  and rt_df['orderId'][0] in df_order['orderId'].values:  # and newClientOrderId in df_order['clientOrderId'].values
                df_order.sort_index(axis=1, ascending=True, inplace=True)
                df_order.loc[ df_order['orderId'] == rt_df['orderId'][0] ] = rt_df.to_numpy()
            else:
                df_order = pd.concat([df_order,rt_df ], ignore_index=True)

            df_o = df_order[['orderId','symbol','price','origQty','status','updateDate','clientOrderId', ] ]

            conn = sqlite3.connect(dbfile)
            # ##df_order = pd.read_sql(' select * from order ', conn) 
            rt_df.to_sql('orders', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_order.to_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )
            # ##df_order = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_order :\n{df_order}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询所有订单+历史 allOrders###订单合并###get###QryAllOrders***** ')  

2023-03-18 07:18:48 *****查询所有订单+历史 allOrders###订单合并###get###QryAllOrders***** 


In [14]:
###查询账户成交历史 userTrades###get###QryUserTrades###
def QryUserTrades(symbol='btcusdt', limit=10):

    global serverUrl, api_key, secret_key, recvWindow, newClientOrderId
    global g_access, df_order, df_trade
    
    timestamp = int(round((time.time() +serverTimeadj) * 1000) )
    url = f'{serverUrl}/fapi/v1/userTrades'

    # symbol = rt['symbol'] if ('rt' in dir() and 'symbol' in rt) else 'btcusdt'
    # orderId = rt['orderId'] if ('rt' in dir() and 'orderId' in rt) else 3204222910

    # startTime = 0  # 起始时间, 查询时间范围最大不得超过7天, 默认查询最近7天内的数据
    # endTime = 0 # 结束时间
    # limit = 10 # 返回的结果集数量 默认值:500 最大值:1000

    query_string = 'symbol=%s&limit=%s&recvWindow=%s&timestamp=%s' % (symbol,limit,recvWindow,str(timestamp))
    signature = hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest() # 用secret_key时间戳hmac sha256加密
    params = {"symbol":symbol,"limit":limit,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
    headers = {"X-MBX-APIKEY":api_key}

    url2 = '%s?%s&signature=%s' % (url, query_string, signature)
    res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
    rt = json.loads(res.text)

    # print(f'{url2=} \n{res.text}')

    # [{"orderId":3216740836,"symbol":"BTCUSDT","status":"FILLED","clientOrderId":"SIB20220926080700MKTbtcusdtGTC0C0A1B","price":"0","avgPrice":"18772.85409","origQty":"0.159","executedQty":"0.159","cumQuote":"2984.88380","timeInForce":"GTC","type":"MARKET","reduceOnly":false,"closePosition":false,"side":"BUY","positionSide":"BOTH","stopPrice":"0","workingType":"CONTRACT_PRICE","priceProtect":false,"origType":"MARKET","time":1664150820741,"updateTime":1664150820741},

    if 'code' in rt and 'msg' in rt:
        logger.error(f'res : {res.text}')
    else :
        for order in rt:
            rt_df = pd.DataFrame([order])
            # print(f"{rt_df = }  ")

            # rt_df['updateDate'] = pd.to_datetime(rt_df['updateTime'], unit="ms") 
            rt_df['updateDate'] = pd.to_datetime(rt_df['time'], unit="ms", origin="1970-01-01 08:00:00") 
            rt_df['queryTime'] = time.strftime("%Y-%m-%d %H:%M:%S")
            # rt_df['time'] = timestamp
            rt_df['access'] = g_access
            
            rt_df.sort_index(axis=1, ascending=True, inplace=True)

            if  not df_trade.empty  and rt_df['id'][0] in df_trade['id'].values:  # 
                df_trade.sort_index(axis=1, ascending=True, inplace=True)
                df_trade.loc[ df_trade['id'] == rt_df['id'][0] ] = rt_df.to_numpy()
            else:
                df_trade = pd.concat([df_trade,rt_df ], ignore_index=True)

            df_tr = df_trade[['id','orderId','symbol','price','qty','side','positionSide','quoteQty', ] ]

            conn = sqlite3.connect(dbfile)
            # ##df_trade = pd.read_sql(' select * from trades ', conn) 
            rt_df.to_sql('trades', con=conn, if_exists='append', index=False) #,
            conn.close()

            df_trade.to_pickle('%s/%s.pkl' % (worthDir, "tradesdf" ) )
            # ##df_trade = pd.read_pickle('%s/%s.pkl' % (worthDir, "orderdf" ) )

        logger.info(f'df_trade :\n{df_trade}')

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询账户成交历史 userTrades###get###QryUserTrades***** ')  

2023-03-18 07:18:51 *****查询账户成交历史 userTrades###get###QryUserTrades***** 


In [17]:
###1. 查询###账户###
QryBalance()
# QryAccount()
# QryUserTrades()

# display(df_balance)
# display(df_account)
# display(df_position)
# display(df_trade)

# TradeFuture() # 交易 限价
# CancelAllOpenOrders()  # 全部撤单
# QryAllOrders(limit=50)  #  查询订单
# QryPositionRisk() # 用户持仓风险V2
# df_posirisk.head(10)
# display(df_order)

###2. 交易###账户###
# TradeFuture() # 交易 限价 单向持仓
# TradeFuture(symbol='btcusdt', side='BUY', quantity=0.001, orderType='MARKET') # 交易 市价 单向持仓

# TradeFuture(symbol='btcusdt', side='BUY', quantity=0.001, orderType='MARKET', positionSide='LONG') # 交易 市价 双向持仓

2023-03-18 07:24:25,867:INFO:sim:323834116.py:17:QryBalance:2332: params: {'recvWindow': '5000', 'timestamp': 1679124265704, 'signature': '3ebcb0a2cb773fa49b3db960f7334c6618b33b1eccba8bdfbf22590f91e62963'} 
 rt: {'code': -2015, 'msg': 'Invalid API-key, IP, or permissions for action, request ip: 4.193.180.41'}
2023-03-18 07:24:25,869:ERROR:sim:323834116.py:20:QryBalance:2332: res: {"code":-2015,"msg":"Invalid API-key, IP, or permissions for action, request ip: 4.193.180.41"}


In [ ]:
conn = sqlite3.connect('../pkl/binance.db')
# df_acc0 = pd.read_sql(' select * from account ', conn) 
# rt_df.to_sql('account', con=conn, if_exists='append', index=False) #,
# df_bal0 = pd.read_sql(' select * from balance ', conn) 
df_ord0 = pd.read_sql(' select * from orders ', conn) 
conn.close()

display(df_ord0)

In [ ]:
cfgfile = f"config.json"
if os.path.exists(cfgfile):
    with open(cfgfile,"rb") as f:
        cfg = json.load(f)  #读取
        print(f"{cfg[cfg['account_type']]['uhost']}")
        print(f"{cfg}")



In [ ]:
# print(f"{ QryServerTime()  }")  # 1663864536221  ms
# print(f"{ datetime.fromtimestamp( QryServerTime() //1000 +28800).strftime('%H%M')  }")  # 1663864536221  ms
# print(f"{ int(datetime.fromtimestamp( 1663114199 //1000 -20000 -1200 ).strftime('%H%M') ) }")  # 1663864536221  ms  -28800

# import inspect
# print(f"{inspect.stack()[1][3] }")
# print(sys._getframe().f_code.co_name)


from dis import dis


quantity = abs(-0.219)
qty = str(quantity).split('.')[0] + '.' + str(quantity).split('.')[1][:3]

print(f"{quantity = } , {qty = } ")

a = 'Hello,world. ByeBye!'
print(a.replace('-',''))

display(df_balance)

In [ ]:
###杠杆分层标准 leverageBracket###get###
import hashlib
import hmac
import requests
import json
from datetime import datetime 
import time

timestamp = int(round((time.time() +serverTimeadj) * 1000) )
recvWindow = '5000'

url = f'{serverUrl}/fapi/v1/leverageBracket'
# api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
# secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

symbol = "btcusdt"

query_string = 'symbol=%s&recvWindow=%s&timestamp=%s' % (symbol,recvWindow,str(timestamp))
secret = secret_key

def hashing(query_string):
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

signature = hashing(query_string) # 用secret_key和毫秒级时间戳进行hmac sha256加密
# print(f'{signature}')
params = {"symbol":symbol,"recvWindow":recvWindow,"timestamp":timestamp,"signature":signature}
headers = {"X-MBX-APIKEY":api_key}

url2 = '%s?%s&signature=%s' % (url, query_string, signature)

res = requests.get(url2, headers=headers) #, params=params  urlTest  url2 
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    logger.info(f'res : {res.text}')

In [10]:
###最新价格 ticker/price###get###QryTcikerPrice###
def QryTcikerPrice(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/ticker/price'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')  
    else :
        logger.info(f'res : {res.text}')  #{"symbol":"BTCUSDT","price":"20137.10","time":1661961001841}

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****最新价格 ticker/price###get###QryTcikerPrice***** ') 

2022-10-08 00:04:20 *****最新价格 ticker/price###get###QryTcikerPrice***** 


In [11]:
###最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex###
def QryPremiumIndex(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/premiumIndex'

    params = {"symbol":symbol, }

    res = requests.get(url, params=params) #
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"symbol":"BTCUSDT","markPrice":"20133.32973555","indexPrice":"20133.75155317","estimatedSettlePrice":"20056.77850121","lastFundingRate":"-0.00194053","interestRate":"0.00010000","nextFundingTime":1661961600000,"time":1661961098000}

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** ') 

2022-10-08 00:04:24 *****最新标记价格和资金费率 premiumIndex###get###QryPremiumIndex***** 


In [12]:
###深度信息 depth###get###QryDepth###
def QryDepth(symbol='btcusdt'):

    url = f'{serverUrl}/fapi/v1/depth'

    params = {"symbol":symbol, "limit":5, } 

    res = requests.get(url, params=params)  
    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt :
        logger.error(f'res : {res.text}')
    else :
        logger.info(f'res : {res.text}')  
        # {"lastUpdateId":27532317617,"E":1664272323243,"T":1664272323241,"bids":[["20173.30","4.937"],["20173.00","0.001"],["20172.90","4.228"],["20172.50","18.436"],["20171.30","14.832"]],"asks":[["20183.40","1.430"],["20183.80","61.862"],["20184.20","128.430"],["20184.60","71.158"],["20185.00","3.459"]]}

        print(f"rt : {rt}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]}")
        print(f"rt['bids'][0][0] : {rt['bids'][0][0]} , rt['asks'][0][0] : {rt['asks'][0][0]}")

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S")} *****###深度信息 depth###get###QryDepth###***** ') 

Now: 1.0.1 : 2022-10-08 00:04:28 *****###深度信息 depth###get###QryDepth###***** 


In [4]:
###查询行情 kline###continuousKlines###
def continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m', startTime=0, endTime=0, limit=499 ):
    
    global serverUrl
    url = f'{serverUrl}/fapi/v1/continuousKlines'

    # pair = 'btcusdt'
    # contractType = 'PERPETUAL'
    # interval = '5m' #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    # startTime = 0
    # endTime = 0
    # limit = 3 #499

    if limit < 0 or startTime < 0 or endTime < 0 or (startTime > endTime):
        k_df = pd.DataFrame()
        logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" or "startTime" or "endTime" is not valid."}}')
        return k_df

    elif limit == 0:
        if startTime == 0 and endTime == 0:
            k_df = pd.DataFrame()
            logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" is not valid."}}') 
            return k_df

        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime }  
    elif limit > 0:
        if startTime == 0 and endTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "limit":limit, }  
        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "limit":limit, }   
        elif startTime > 0 and endTime > 0:  # startTime + limit < endTime
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "endTime":endTime, "limit":limit, }   

    logger.info(f'{params=}')
    
    res = requests.get(url, params=params)

    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        k_df = pd.DataFrame()
        logger.error(f'res : {res.text}')
    else:
        k_df = pd.DataFrame(rt)
        if k_df.empty:
            return k_df    
        
        k_df.columns = ['openTime','open','high','low','close','vol','closeTime','Amt','numTrade','bidVol','bidAmt','ignore'] #
        
        k_df['open']  = pd.to_numeric(k_df['open'] , errors='coerce', downcast='float')
        k_df['high']  = pd.to_numeric(k_df['high'] , errors='coerce', downcast='float')
        k_df['low']  = pd.to_numeric(k_df['low'] , errors='coerce', downcast='float')
        k_df['close']  = pd.to_numeric(k_df['close'] , errors='coerce', downcast='float')
        k_df['vol']  = pd.to_numeric(k_df['vol'] , errors='coerce', downcast='float')
        k_df['Amt']  = pd.to_numeric(k_df['Amt'] , errors='coerce', downcast='float')
        k_df['bidVol']  = pd.to_numeric(k_df['bidVol'] , errors='coerce', downcast='float')
        k_df['bidAmt']  = pd.to_numeric(k_df['bidAmt'] , errors='coerce', downcast='float')
        k_df['openTime']  = pd.to_numeric(k_df['openTime'] , errors='coerce', downcast='integer')
        k_df['closeTime']  = pd.to_numeric(k_df['closeTime'] , errors='coerce', downcast='integer')
        k_df['numTrade']  = pd.to_numeric(k_df['numTrade'] , errors='coerce', downcast='integer')

        k_df['openDateTime'] = pd.to_datetime(k_df['openTime'], unit="ms") #, origin="1970-01-01 08:00:00" , unit="ms"
        k_df['closeDateTime'] = pd.to_datetime(k_df['closeTime']//1000, unit="s") 
        # k_df['time'] = pd.to_datetime(int(time.time()), unit='s', unorigin="1970-01-01 08:00:00")
        
        k_df.drop(['ignore'], axis=1, inplace=True)

        k_df.sort_index(axis=1, ascending=True, inplace=True)

    return k_df

#权重 [1,100)  1 ; [100, 500)	2 ; [500, 1000]	5 ;  > 1000	10 ;

# [
# 1661744100000,      // 开盘时间 openTime
# "18879.99",         // 开盘价 open
# "18900.00",         // 最高价 high
# "18878.98",         // 最低价 low
# "18896.13",         // 收盘价(当前K线未结束的即为最新价) close
# "492.363",          // 成交量 vol
# 1661744399999,      // 收盘时间 closeTime
# "9302145.66080",    // 成交额 Amt
# 1874,               // 成交笔数 numTrade
# "385.983",          // 主动买入成交量 bidVol
# "7292402.33267",    // 主动买入成交额 bidAmt
# "0"                 // 请忽略该参数 ignore
# ]

# k_df = continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(limit=2, endTime =1661824559999 ) # 1661824559999 ,2 
# k_df = continuousKlines(limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(startTime =1661824259000, endTime =1661824559999 ) # 5
# k_df = continuousKlines(limit=2, startTime =1661824259000, ) # 2
# k_df = continuousKlines(limit=0, startTime =0, endTime =0 ) #err 0 
# k_df = continuousKlines(limit=-2 ) # err 0
# k_df = continuousKlines(limit=2 ) # 2
# print(f'{k_df}')  #11s

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询行情 kline###continuousKlines***** ')

2022-10-25 02:40:52 *****查询行情 kline###continuousKlines***** 


In [5]:
###查询历史行情 kline###histUKline###
def histUKline(symbol="BTCUSDT", interval="1m", histDays=1, limit=998):
    
    global locTimeadj

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]
    curSec = int(time.time())
    startSec = int(time.mktime(time.strptime(time.strftime("%Y%m%d", time.localtime(time.time()-60*60*24*histDays ) ), '%Y%m%d')))
    startSec = startSec -locTimeadj -1

    interval_pre = interval
    if interval_pre == '10m':
        interval = '5m'
        intervalSec = intervalSec//2

    totalCount = math.ceil( (curSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {curSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, curSec, intervalSec*limit):

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000+999=}', ', endTime=', f'{(Sec+intervalSec*limit-1)*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000+999, endTime=(Sec+intervalSec*limit-1)*1000+999 )

        kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,kdf],ignore_index=True)
        del kdf

        count += 1
        time.sleep(1)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    if interval_pre == '10m':
        interval = interval_pre
        intervalSec = int(intervalSec*2)
        rePeriod = '10T'

        ukdfHistCp = ukdfHist.copy()
        ukdfHistCp['dateTime'] = ukdfHistCp['closeSec'].apply(lambda x: datetime.fromtimestamp(x))
        ukdfHistCp = ukdfHistCp.set_index(keys=['dateTime'], drop=True)
        # ukdfHistCp = ukdfHistCp.reindex(ukdfHistCp['dateTime'].sort_values(ascending=True).index)

        logger.info(f'ukdfHistCp.iloc[:5,:] :\n{ukdfHistCp.iloc[:5,:]}')

        if not ukdfHistCp.empty:
            openSr = ukdfHistCp['open'].resample(rePeriod, label='right').first()  
            openTimeSr = ukdfHistCp['openTime'].resample(rePeriod, label='right').first()  
            closeSr = ukdfHistCp['close'].resample(rePeriod, label='right').last()  #last first max min
            closeTimeSr = ukdfHistCp['closeTime'].resample(rePeriod, label='right').last()  
            closeSecSr = ukdfHistCp['closeSec'].resample(rePeriod, label='right').last()  
            tradeDateSr = ukdfHistCp['tradeDate'].resample(rePeriod, label='right').last()  

            ukdf = pd.concat([tradeDateSr, openTimeSr, closeTimeSr, closeSecSr,openSr, closeSr], axis=1) 
            
        ukdf.columns = ['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close']
        ukdf.reset_index(drop=True,inplace=True)

        if ukdf.iloc[-1]['closeSec'] - ukdf.iloc[-2]['closeSec'] < intervalSec:
            ukdf.drop([len(ukdf)-1],inplace=True)

        # ukdf = ukdfHistCp.copy()
        # logger.info(f'{ukdf.iloc[:5,:]=}')
        
        del ukdfHistCp

    else:
        ukdf = ukdfHist.copy()

    del ukdfHist

    ukdf['close'] = ukdf['close'].astype(float)
    ukdf['pct'] = ukdf['close']/ukdf['close'].shift(1)-1  #涨跌幅
    ukdf.fillna(0, inplace=True)

    logger.info(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
    logger.info(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

    return ukdf

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询历史行情 kline###histUKline***** ')

2022-10-25 02:40:59 *****查询历史行情 kline###histUKline***** 


In [6]:
###查询历史行情 日kline###getDayUKline###
def getDayUKline(symbol="BTCUSDT", interval="1m", date="0", histDays=1, limit=499):
    
    if date == "0":
        startSec = int(datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%s"))
    else:
        startSec = int(datetime.strptime(date, "%Y%m%d").strftime("%s"))
    
    endSec = int( ( datetime.fromtimestamp(startSec)+ timedelta(days=histDays) ).strftime("%s") )

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]

    totalCount = math.ceil( (endSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {endSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, endSec, intervalSec*limit):

        stopSec = min(Sec+intervalSec*limit-1 , endSec-1)

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000=}', ', endTime=', f'{stopSec*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000, endTime=stopSec*1000+999 )

        # kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,ukRet],ignore_index=True)
        # del kdf

        count += 1
        # time.sleep(1)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    return ukdfHist

print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+locTimeadj))} *****查询历史行情 日kline###getDayUKline***** ')

2022-10-25 02:41:06 *****查询历史行情 日kline###getDayUKline***** 


In [16]:
def getSimilarity(ukdf, wid) -> pd.DataFrame:

    ukdfT = ukdf.pivot(index='closeTime', columns='tradeDate', values='close')
    ukdfAmT = ukdfT.iloc[:wid].copy()
    ukdfAmT.dropna(how='any',inplace=True)
    ukdfAmT.reset_index(drop=True,inplace=True)
    ukdfAmT = ukdfAmT.apply(lambda x: x.astype(float))

    logger.info(f"{ukdfAmT.corr()=}")

    df_corr = ukdfAmT.corr().iloc[:, [-1]]
    df_corr.columns=['corr']
    # df_corr = df_corr.reindex(df_corr['corr'].abs().sort_values(ascending=False).index)
    df_corr = df_corr.reindex(df_corr['corr'].sort_values(ascending=False).index)

    logger.info(f"df_corr: {df_corr}")  # corr: df_corr.iloc[1,0], data: df_corr.index[1]
    return df_corr


In [7]:
symbol = "BTCUSDT"
interval = "10m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
histDays = 2  # 30 10 60 84 120 
limit = 998 # 1500  499 998
wid = 42

ukdf = histUKline( interval=interval, histDays=histDays, limit=498)
print(ukdf.shape)
# print(ukdf)
# print(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
# print(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

# ukRet = continuousKlines(pair=symbol, interval="5m", limit=3 )
# kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])
# print(kdf)

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# t1.start()
# stop_thread(thread)

# nohup python3 -u main.py -n testStrategy -s 8808 -c 29090 -X BTCUSDT -p 10m -w 42 -d 84 -t 1000000  >> log.txt 2>&1  &


2022-10-25 02:41:36,651:INFO:sim:1093784808.py:18:histUKline:31528: interval='5m', startSec=1666483199, curSec=1666665696, intervalSec=300, limit=498, intervalSec*limit=149400, totalCount=609, histDays=2
count=1: continuousKlines( symbol='BTCUSDT' , interval='5m' , limit=498 , Sec*1000+999=1666483199999 , endTime= 1666632598999  )
2022-10-25 02:41:36,655:INFO:sim:3176104500.py:39:continuousKlines:31528: params={'pair': 'BTCUSDT', 'contractType': 'PERPETUAL', 'interval': '5m', 'startTime': 1666483199999, 'endTime': 1666632598999, 'limit': 498}
count=2: continuousKlines( symbol='BTCUSDT' , interval='5m' , limit=498 , Sec*1000+999=1666632599999 , endTime= 1666781998999  )
2022-10-25 02:41:37,935:INFO:sim:3176104500.py:39:continuousKlines:31528: params={'pair': 'BTCUSDT', 'contractType': 'PERPETUAL', 'interval': '5m', 'startTime': 1666632599999, 'endTime': 1666781998999, 'limit': 498}
2022-10-25 02:41:39,075:INFO:sim:1093784808.py:39:histUKline:31528: ukdfHist.iloc[:5,:] :
  tradeDate open

In [19]:
# print( ukdf.loc[ukdf.index[-1],'close']  )

startTime = int(f"{ukdf.loc[ukdf.index[-2],'closeSec']}000")
endTime = int(time.time() * 1000)
print( ukdf.loc[ukdf.index[-2],'closeSec'] , startTime, endTime )

1664888999 1664888999000 1664889624608


In [20]:
ukRet = continuousKlines(pair=symbol, interval="5m", startTime=startTime, endTime=endTime )  # limit=4

kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000+locTimeadj) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000+locTimeadj) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000+locTimeadj) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

print(kdf)

2022-10-04 21:20:26,799:INFO:sim:3176104500.py:39:continuousKlines:124561: params={'pair': 'BTCUSDT', 'contractType': 'PERPETUAL', 'interval': '5m', 'startTime': 1664888999000, 'endTime': 1664889624608, 'limit': 499}
  tradeDate openTime closeTime    closeSec     open    close
0  20221004   211000    211459  1664889299  19984.9  20026.3
1  20221004   211500    211959  1664889599  19996.2  20025.2
2  20221004   212000    212459  1664889899  20025.2  20021.1


In [21]:
# print(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')
print(kdf)
print(type( ukdf.loc[ukdf.index[-1],'close'] ) )
print(type( kdf.loc[kdf['closeSec']==ukdf['closeSec'].iloc[-1],'close'].iloc[-1] ) )
ukdf.loc[ukdf.index[-1],'close'] = kdf.loc[kdf['closeSec']==ukdf['closeSec'].iloc[-1],'close'].iloc[-1]
ukdf.loc[ukdf.index[-1],'pct'] = ukdf['close'].iloc[-1]/ukdf['close'].iloc[-2]-1  #涨跌幅

print(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

  tradeDate openTime closeTime    closeSec     open    close
0  20221004   211000    211459  1664889299  19984.9  20026.3
1  20221004   211500    211959  1664889599  19996.2  20025.2
2  20221004   212000    212459  1664889899  20025.2  20021.1
<class 'numpy.float64'>
<class 'numpy.float64'>
ukdf.iloc[-5:,:] :
    tradeDate openTime closeTime    closeSec     open    close       pct
411  20221004   203000    203959  1664887199  19950.8  19920.0 -0.001258
412  20221004   204000    204959  1664887799  19929.3  19973.5  0.002686
413  20221004   205000    205959  1664888399  19933.7  19978.8  0.000265
414  20221004   210000    210959  1664888999  19974.0  19984.9  0.000305
415  20221004   211000    211959  1664889599  19984.9  20025.2  0.002017


In [ ]:
corrdf = getSimilarity(ukdf,wid)
print(f'\n ukdf: \n{ukdf} \n')
print(f'\n corrdf: \n{corrdf} \n')

In [22]:
###1. 查询###余额 balance###
QryBalance()
QryAccount()
display(df_balance)
display(df_account)
display(df_position)


2022-10-06 18:16:32,531:INFO:sim:638711108.py:17:QryBalance:148919: params: {'recvWindow': '5000', 'timestamp': 1665051392430, 'signature': 'f575954df5686dc482aebd2bb43a0ae168a26bcec013c07e96091178d403f339'} 
 rt: [{'accountAlias': 'oCAuFzSgAumY', 'asset': 'DOT', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', 'asset': 'BTC', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', 'asset': 'SOL', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', 'a

,accountAlias,asset,availableBalance,balance,crossUnPnl,crossWalletBalance,marginAvailable,maxWithdrawAmount,queryTime,updateDate,updateTime
0,oCAuFzSgAumY,USDT,905.0,1000.0,0.0,1000.0,True,905.0,2022-10-06 17:44:53,2022-10-06 13:08:09.558,1665032889558
1,oCAuFzSgAumY,USDT,1000.0,1000.0,0.0,1000.0,True,1000.0,2022-10-06 18:16:32,2022-10-06 13:08:09.558,1665032889558


In [22]:
###1. 查询###余额 balance###
QryBalance()
print(f'\n df_balance: \n{df_balance} \n')  #
# print(f'{df_balance.iloc[-1, df_balance.columns.get_loc("availableBalance")]} , {df_balance.loc[df_balance.index[-1],"balance"]} ')

###2. 查询账户数据 account###余额assets+持仓positions###QryAccount###
QryPremiumIndex()
QryAccount()

print(f'\n df_account: \n{df_account} \n')
print(f'\n df_position: \n{df_position} \n')

###3. 查询订单 orders###订单合并###
# QryOrder(orderId)  #3205875652  orderId
# QryOpenOrders()

###4. 用户持仓风险V2 positionRisk###get###QryPositionRisk###
QryTcikerPrice()
# QryPositionRisk()



2022-10-06 17:32:02,262:INFO:sim:4262489530.py:17:QryBalance:148421: params: {'recvWindow': '5000', 'timestamp': 1665048722160, 'signature': 'a45728627ce250f817e68a1289b81396270b6f5c31b53d3f7090743d6ae363f5'} 
 rt: [{'accountAlias': 'oCAuFzSgAumY', 'asset': 'DOT', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', 'asset': 'BTC', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', 'asset': 'SOL', 'balance': '0.00000000', 'crossWalletBalance': '0.00000000', 'crossUnPnl': '0.00000000', 'availableBalance': '0.00000000', 'maxWithdrawAmount': '0.00000000', 'marginAvailable': True, 'updateTime': 0}, {'accountAlias': 'oCAuFzSgAumY', '

In [23]:
###3. 查询订单 orders###订单合并###
# QryOrder(orderId)  #3205875652  orderId
QryOpenOrders()

2022-10-06 17:34:07,265:INFO:sim:2537277463.py:60:QryOpenOrders:148421: df_order :
   avgPrice                 clientOrderId  closePosition  cumQuote  executedQty      orderId  origQty origType positionSide    price  priceProtect  reduceOnly side status  stopPrice   symbol                time timeInForce   type              updateDate     updateTime     workingType
0       0.0  android_8CMDA2gkafbYNMmiZFlL          False       0.0          0.0  81444432962    0.005    LIMIT         BOTH  19000.0         False       False  BUY    NEW        0.0  BTCUSDT 2022-10-06 17:34:07         GTC  LIMIT 2022-10-06 17:32:40.841  1665048760841  CONTRACT_PRICE


In [24]:
display(df_order)

,avgPrice,clientOrderId,closePosition,cumQuote,executedQty,orderId,origQty,origType,positionSide,price,priceProtect,reduceOnly,side,status,stopPrice,symbol,time,timeInForce,type,updateDate,updateTime,workingType
0,0.0,android_8CMDA2gkafbYNMmiZFlL,False,0.0,0.0,81444432962,0.005,LIMIT,BOTH,19000.0,False,False,BUY,NEW,0.0,BTCUSDT,2022-10-06 17:34:07,GTC,LIMIT,2022-10-06 17:32:40.841,1665048760841,CONTRACT_PRICE


In [19]:
display(df_balance)
print("\n")
display(df_account)
print("\n")
display(df_position)



,accountAlias,asset,availableBalance,balance,crossUnPnl,crossWalletBalance,marginAvailable,maxWithdrawAmount,queryTime,updateDate,updateTime
0,oCAuFzSgAumY,BTC,0.0,0.0,0.0,0.0,True,0.0,2022-10-06 16:46:13,1970-01-01 08:00:00,0


,asset,availableBalance,crossUnPnl,crossWalletBalance,initialMargin,maintMargin,marginAvailable,marginBalance,maxWithdrawAmount,openOrderInitialMargin,positionInitialMargin,queryTime,unrealizedProfit,updateDate,updateTime,walletBalance
0,USDT,1000.0,0.0,1000.0,0.00000000,0.0,True,1000.0,1000.0,0.0,0.0,2022-10-06 16:46:13,0.0,2022-10-06 13:08:09.558,1665032889558,1000.0


""


In [22]:
QryTcikerPrice()
QryPremiumIndex()

2022-10-06 11:29:49,238:INFO:sim:1778055962.py:14:QryTcikerPrice:145193: res : {"symbol":"BTCUSDT","price":"20345.70","time":1665026988004}
2022-10-06 11:29:49,333:INFO:sim:4138418679.py:14:QryPremiumIndex:145193: res : {"symbol":"BTCUSDT","markPrice":"20346.47622419","indexPrice":"20349.99273443","estimatedSettlePrice":"20325.19108336","lastFundingRate":"0.00010000","interestRate":"0.00010000","nextFundingTime":1665043200000,"time":1665026989000}


In [ ]:
df = getDayUKline( date="20220913")

print(f'df.iloc[:5,:] :\n{df.iloc[:5,:]}')
print(f'df.iloc[-5:,:] :\n{df.iloc[-5:,:]}')

print(df.shape)

conn = sqlite3.connect(dbfile)
# df_btcusdt_1m = pd.read_sql(' select * from btcusdt_1m ', conn) 
df.to_sql('btcusdt_1m', con=conn, if_exists='append', index=False) #,
conn.close()

In [3]:
df = pd.DataFrame(np.array([i for i in range(1,41)]).reshape(10,4), columns=['A','B','C','D'])

In [4]:
print(f"{df}")
df['D'].iloc[-1] = 100
print(f"{df}")
df.iloc[-1, df.columns.get_loc('C')] = 101
print(f"{df}")

    A   B   C   D
0   1   2   3   4
1   5   6   7   8
2   9  10  11  12
3  13  14  15  16
4  17  18  19  20
5  21  22  23  24
6  25  26  27  28
7  29  30  31  32
8  33  34  35  36
9  37  38  39  40
    A   B   C    D
0   1   2   3    4
1   5   6   7    8
2   9  10  11   12
3  13  14  15   16
4  17  18  19   20
5  21  22  23   24
6  25  26  27   28
7  29  30  31   32
8  33  34  35   36
9  37  38  39  100
    A   B    C    D
0   1   2    3    4
1   5   6    7    8
2   9  10   11   12
3  13  14   15   16
4  17  18   19   20
5  21  22   23   24
6  25  26   27   28
7  29  30   31   32
8  33  34   35   36
9  37  38  101  100
